In [1]:
%run dataFormating.ipynb

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

# What subsets of scientific questions tend to be answered correctly by the same subjects?

## Mining

In [6]:
from orangecontrib.associate.fpgrowth import *  
import pandas as pd
from numpy import *

In [18]:
itemsets = frequent_itemsets(array(correctedScientific), 10)
list(itemsets)[:5]

[(frozenset({0}), 59),
 (frozenset({18}), 52),
 (frozenset({0, 18}), 47),
 (frozenset({14}), 44),
 (frozenset({0, 14}), 40)]

In [15]:
rules = association_rules(dict(itemsets), .8)
rules = list(rules)
len(rules)

0

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Get frequent itemsets with support > 5%
# run time < 15 min
partial = correctedScientific.iloc[:, 0:18] #For now can only compute the rules for 18 questions
frequent_itemsets = apriori(partial, min_support=0.25, use_colnames=True)
frequent_itemsets.shape
#frequent_itemsets

In [ ]:
# Generate rules according to confidence
# < run time < 
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
rules.head()

In [ ]:
# Save the mined rules to file
#rules.to_csv("results/associationRulesMiningSupport25percentsConfidence90percents.csv")

## Search for interesting rules
Interesting rules are more likely to be the ones with highest confidence, the highest lift or with a bigger consequent set. Pairs can also be especially interesting

In [ ]:
# Sort rules by confidence and lift
confidenceSortedRules = rules.sort_values(by = ["confidence", "lift"], ascending=[False, False])
confidenceSortedRules.head(50)

In [ ]:
# Sort rules by size of consequent set
rules["consequentSize"] = rules["consequents"].apply(lambda x: len(x))
consequentSortedRules = rules.sort_values(by = ["consequentSize", "confidence", "lift"], ascending=[False, False, False])
consequentSortedRules.head(50)

In [ ]:
# Select only pairs (rules with antecedent and consequent of size one)
# Sort pairs according to confidence
rules["fusedRule"] = rules[["antecedants", "consequents"]].apply(lambda x: frozenset().union(*x), axis=1)
rules["ruleSize"] = rules["fusedRule"].apply(lambda x: len(x))
pairRules = rules.sort_values(by=["ruleSize", "confidence", "lift"], ascending=[True, False, False])
pairRules.head(30)

In [ ]:
# Sort questions by number of apparition in consequents
questions = list(correctedScientific.columns)
for q in questions:
    rules[q+"c"] = rules["consequents"].apply(lambda x: 1 if q in x else 0)
occurenceInConsequents = rules.loc[:,"Q1c":"Q27c"].sum(axis=0)
occurenceInConsequents

In [ ]:
# Sort questions by number of apparition in antecedants
for q in questions:
    rules[q+"a"] = rules["antecedants"].apply(lambda x: 1 if q in x else 0)
occurenceInAntecedants = rules.loc[:,"Q1a":"Q27a"].sum(axis=0)
occurenceInAntecedants